<a href="https://colab.research.google.com/github/MohsenJadidi/Automatic-Modulation-Classification-AMC/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Importing the dataset (Copy from CNN.ipynb and merge cells)

In [13]:
import pickle
import numpy as np
fileName = 'RML2016.10a_dict.pkl'
with open("/content/drive/My Drive/Colab Notebooks/"+fileName,'rb') as f:
  data = pickle.load(f,encoding='bytes')
 
X = []
labels = [] # label each example by a pair (modulation type, snr)
total_examples = 0
analog = [b'AM-DSB', b'AM-SSB', b'WBFM']

for mod_type, snr in data.keys():
    if (mod_type not in analog):      
        current_matrix = data[(mod_type, snr)]        
        total_examples += current_matrix.shape[0]
        for i in range(current_matrix.shape[0]):
            X.append(current_matrix[i])
            labels.append((str(mod_type, 'ascii'), snr)) # mod_type is of type bytes
    
X = np.array(X)         # First row is QPSK snr=2, seconde is PAM4 snr=8 , ...
labels = np.array(labels)

y = labels[:,0]

print(f'loaded {total_examples} signal vectors into X{X.shape} and their corresponding'
      f' labels into labels{labels.shape}')  
# print(np.unique(labels[:,0]))

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
onehotencoder = OneHotEncoder()
y = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()

snrList = [str(2*i-20) for i in range(20)]  # snrList = -20, -18, -16 , ... ,0, ... ,18
snr = snrList[19]
numberOfEachExamples = 1000
print("SNR :", snr)

output = [[labels[i*numberOfEachExamples, 0],y[i*numberOfEachExamples]] for i in range(int(X.shape[0]/numberOfEachExamples))]
output = dict(output)

X_snr = []
y_snr = []

for i in range(X.shape[0]):
    if labels[i,1] == snr:
        X_snr.append(X[i])
        y_snr.append(y[i])

X_snr = np.array(X_snr)
y_snr = np.array(y_snr)  

###### Splitting the dataset into the Training set and Test set ######
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_snr, y_snr, test_size = 0.2, random_state = 0)
# The below line better for Cross_val part
#X_train, X_test, y_train, y_test = train_test_split(X_snr, y_snr, test_size = 1, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# StandardScaler expected <= 2.
#X_train = X_train.reshape([6400,256])
X_train = X_train.reshape([X_train.shape[0],256])
X_train = sc.fit_transform(X_train)
X_train = X_train.reshape([X_train.shape[0],2,128])
#X_test = X_test.reshape([1600,256])
X_test = X_test.reshape([X_test.shape[0],256])
X_test = sc.transform(X_test)
X_test = X_test.reshape([X_test.shape[0],2,128])

# Reshape
X_train = X_train.reshape(-1,2, 128, 1)   #Reshape for CNN -  (6400,2,128)->(6400,2,128,1)!!
X_test = X_test.reshape(-1,2, 128, 1)

loaded 160000 signal vectors into X(160000, 2, 128) and their corresponding labels into labels(160000, 2)
SNR : 18


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Import Module

In [0]:
import keras
from keras.models import Model, Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input

#Making ResNet0 model (four-convolutional-layer)

In [14]:
dout = 0.4

modelInput = Input(shape=(2,128,1))

x1 = ZeroPadding2D((0, 2)) (modelInput)
x1 = Conv2D(256,1,3, activation = 'relu')(x1)
x1 = Dropout(rate=dout)(x1)

x2 = Conv2D(256,2,3, activation = 'relu')(x1)
x2 = Dropout(rate=dout)(x2)

skip1 = add([x2,modelInput])

x3 = Conv2D(80,1,3, activation = 'relu')(skip1)
x3 = Dropout(rate=dout)(x3)

x4 = Conv2D(80,1,3, activation = 'relu')(x3)
x4 = Dropout(rate=dout)(x4)

x5 = Flatten()(x4)

x6 = Dense(output_dim = 128 , activation = 'relu')(x5)
x6 = Dropout(rate=dout)(x6)

out = Dense(output_dim = 8 , activation = 'softmax')(x6)

classifier = Model(inputs = modelInput, outputs = out)

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (2, 3), activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (1, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (1, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softma

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 2, 128, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 2, 132, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 2, 130, 256)  1024        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 2, 130, 256)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
from keras.layers.core import Reshape
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers import Concatenate,Input,add

in_shp = list(X_train.shape[1:])
dr = 0.5


model = Sequential()  # 这里使用keras的序贯模型  https://keras-cn.readthedocs.io/en/latest/models/sequential/
# Reshape [N,2,128] to [N,1,2,128] on input
input_x = Input(shape=(2, 128)) 

model0=(Reshape(([1] + in_shp), input_shape=in_shp))(input_x)
model0=(ZeroPadding2D((0, 2), data_format="channels_first"))(model0)
print(model0)
# 卷积核维度(输出尺度),卷积核的宽度和长度,“valid”代表只进行有效的卷积，即对边界数据不处理,
# 层权重weights的初始化函数
# channels_first corresponds to inputs with shape (batch, channels, height, width).

model1=(Conv2D(256, (1, 3), padding='same', activation="relu", name="conv1", init='glorot_uniform',
                 data_format="channels_first"))(model0)
model1=(Dropout(dr))(model1)
#model1=(ZeroPadding2D((0, 2), data_format="channels_first"))(model1)

model2=(Conv2D(256, (2, 3), padding="same", activation="relu", name="conv2", init='glorot_uniform',
                 data_format="channels_first"))(model1)
#model2=(ZeroPadding2D((0, 2), data_format="channels_first"))(model2)
modela = add([model1, model2])
model3=(Conv2D(80, (1, 3), padding='same', activation="relu", name="conv3", init='glorot_uniform',
                 data_format="channels_first"))(modela)
#model3=(ZeroPadding2D((0, 2), data_format="channels_first"))(model3)

#model3=(Dropout(dr))(model3)
model4 = Concatenate(axis = 1)([model1, model2,model3])

model4=(Conv2D(80, (1, 3), padding='valid', activation="relu", name="conv4", init='glorot_uniform',
                 data_format="channels_first"))(model4)
# 多维的输入一维化，常用在从卷积层到全连接层的过渡

x = (Flatten())(model4)
x=(Dense(128, activation='relu',  name="dense1"))(x)
x=(Dropout(dr))(x)
x=(Dense(8, activation = 'softmax' ,name="dense2"))(x)

x=(Reshape([8]))(x)
model = Model(inputs=input_x, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Set up some params
epochs = 25  # number of epochs to train on
batch_size = 64  # training batch size default1024

#model.load_weights('dense.h5')

filepath = "dense.h5"   # 所要保存的文件名字，h5格式，不用写路径，默认在程序执行的文件夹内
history = model.fit(X_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

Train on 6400 samples, validate on 1600 samples
Epoch 1/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1699 - acc: 0.9231 - val_loss: 0.6225 - val_acc: 0.7944
Epoch 2/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1650 - acc: 0.9267 - val_loss: 0.5979 - val_acc: 0.7994
Epoch 3/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1651 - acc: 0.9281 - val_loss: 0.6374 - val_acc: 0.8025
Epoch 4/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1550 - acc: 0.9295 - val_loss: 0.6222 - val_acc: 0.7987
Epoch 5/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1560 - acc: 0.9294 - val_loss: 0.6052 - val_acc: 0.7913
Epoch 6/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1653 - acc: 0.9250 - val_loss: 0.6614 - val_acc: 0.7812
Epoch 7/25
6400/6400 [==============================] - 7s 1ms/step - loss: 0.1502 - acc: 0.9338 - val_loss: 0.6753 - val_acc: 0.7831
Epoch 8/25
640

# Fitting model

In [0]:
batch = 16
epoch = 25
#history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch, validation_data=(X_test, y_test))
history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch)

Epoch 1/25
6400/6400 [==============================] - 8s 1ms/step - loss: 1.9819 - acc: 0.2008
Epoch 2/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.8493 - acc: 0.2289
Epoch 3/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.7924 - acc: 0.2397
Epoch 4/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.7401 - acc: 0.2497
Epoch 5/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.7141 - acc: 0.2539
Epoch 6/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.7116 - acc: 0.2564
Epoch 7/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.6949 - acc: 0.2728
Epoch 8/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.6667 - acc: 0.2689
Epoch 9/25
6400/6400 [==============================] - 7s 1ms/step - loss: 1.6620 - acc: 0.2884
Epoch 10/25
3760/6400 [================>.............] - ETA: 2s - loss: 1.6285 - acc: 0.3199